### 1. Introduction

We consider an open decentralized learning experiment conducted by a number of peers. Some peers may be malicious attackers, and we assume that their share of the total computational power is $\delta < 50\% - \epsilon$.

A peer $p$ is identified by a pair of a public and private key $(k_p, \hat k_p)$.

During the experiment, the peers calculate gradients for batches. We assume that all batches are of the same size (if a peer have more computational power, it can pretend to be several peers and process several batches simultaneously). Occasionally peers want to synchronize their calculations and average the gradients. During this procedure, peers conduct statistical tests to ensure that another peer haven't sent a spoiled value.

If the peer $p$ (plantiff) detects that the peer $i$ (indicted) has sent a spoiled value, it concludes that $i$ is an attacker and wants to announce the message $accuse(p, i)$ to the network.

We can't believe each $accuse(p, i)$ message and automatically ban $i$ (otherwise an attacker may accuse all other peers to disrupt the experiment). Also, it is computationally impractical to validate all such messages. Therefore, we want to create a scheme for processing the $accuse(p, i)$ messages such that while $\delta < 50\% - \epsilon$, the experiment can't be disrupted.

### 2. The idea

#### 2.1. Overview

We propose a scheme where announcing $accuse(p, i)$:

- Excludes gradients from both $p$ and $i$ from averaging (we can't recognize who of them lies)
- Temporarily bans $p$ and $i$, so that they can't immediately rejoin with other IDs to continue sending wrong gradients/false accusations

Before joining the network, all peers are required to calculate $ POW(k_p, seed) $, where:

- $POW$ (proof of work) is a function that is difficult to calculate but whose results are easy to verify
- $seed$ is a random number

A valid POW is required for sending gradients and calling $accuse(p, i)$. If a peer sends wrong gradients or accuses someone, it loses its POW (it is added to a DHT-based blacklist).

#### 2.2. Estimating computing power to neutralize $\delta$ attackers

Let $R_{pow}$ be amount of computations required to calculate the POW (roughly the number of floating point operations), $R_{batch}$ be amount of computations for a batch's gradients. We select such a POW that $R_{pow} = \alpha R_{batch}$ (usually $\alpha > 1$).

In this case:

1. When the peer $i$ sends wrong gradients and $p$ accuses it, $i$ wastes $ \ge R_{pow} $ computations (loses its POW) and $p$ wastes $ R_{pow} + R_{batch} = \frac{\alpha + 1}\alpha R_{batch} $ computations (loses its POW and last gradients).

2. When $i$ falsely accuses $p$, the peer $i$ wastes $ R_{pow} $ computations and $p$ wastes $ R_{pow} + R_{batch} = \frac{\alpha + 1}\alpha R_{batch} $ computations as well.

To neutralize malicious share $\delta$ of total network computations, we need share $ \frac{\alpha + 1}\alpha \delta$ of total computations made by honest peers. The remaining share $1 - \frac{2 \alpha + 1}\alpha \delta$ of total computations will continue to contribute to the experiment.

This means that while $\delta < \delta_{max} = \frac{\alpha}{2 \alpha + 1}$, the network is able to survive the attack and continue learning (though with slower speed/smaller batches). Note that $\delta_{max}$ approaches $\frac{1}{2}$ with increasing the POW difficulty $\alpha$ (i.e. the delay for nodes to join the network).

In practice, if we assume that calculating $R_{batch}$ takes 30 seconds (reasonable for hivemind) and choose $\alpha = 6$, we can tolerate up to $\delta_{max} \approx 46.2\%$ malicious computing power with the reasonable 3-minute delay for nodes to calculate the POW before joining.

### 3. The protocol

#### 3.1. Sending gradients

The sender $s$ sends the following message signed with its private key $\hat k_s$:

$(\text{gradients}, \, k_s, \, seed, \, POW(k_s, seed))$

The receiver validates that:

1. $POW(k_s, seed)$ is solved correctly
2. $POW(k_s, seed)$ is not in the DHT-based blacklist (see below)
3. The signature is correct w.r.t. $k_s$ (so the sender didn't use someone else's POW)

If the validation fails, the receiver ignores the message. Otherwise, the receiver proceeds to running the statistical checks and **accuses** the sender if they fail.

#### 3.2. Accusing a peer

If a peer $p$ accuses $i$, both $POW(k_p, seed_p)$ and $POW(k_i, seed_i)$ are blacklisted.

Technically, accusing someone corresponds to adding two $(key, \, subkey, \, value)$ tuples singed with the private key $\hat k_p$ to the DHT:

1. $record_1 = (\text{"blacklist_"} \, POW(k_p, seed_p), \quad k_p, \quad \text{""})$
2. $record_2 = (\text{"blacklist_"} \, POW(k_i, seed_i), \quad k_p, \quad \text{serialize}(\text{sign}(record_1, \hat k_p)))$

Both records must have the expiration time $+\infty$ (to be stored forever).

Accusing someone proceeds as follows (we assume $n_1$ and $n_2$ to be honest DHT nodes here, see below for the other case):

1. The peer $p$ asks a DHT node $n_1$ to store $record_2$.

2. $n_1$ validates the format of outer and inner records, the signatures and the POWs.

3. $n_1$ extracts the signed $record_1$ and asks a DHT node $n_2$ to store the inner $record_1$.

4. $n_2$ validates the format of the record, the signatures and the POWs. It agrees to store the record only if $p$ is indeed blacklisting his own.

5. $n_2$ checks whether the $POW(k_p, seed_p)$ is already blacklisted (whether the key already exists). If it is, the POW is wasted and $n_2$ returns $\text{store_ok = False}$ (otherwise, $\text{store_ok = True}$).

6. $n_1$ agrees to save $record_2$ (= ban someone else's POW) only if $n_2$ returned $\text{store_ok = True}$.

Checking whether someone is in the blacklist proceeds as follows:

1. The peer $r$ asks a DHT node for the key $ \text{"blacklist_"} \, POW(k_s, seed_s) $.

<!-- 2. It validates the key format, the POW and the signatures. If $s$ didn't ban itself, $r$ asks another DHT node for the key $ \text{"blacklist_"} \, POW(k_p, seed_p) $ to ensure that  -->

#### 3.3. Hardening the DHT

The scheme above works for small $\delta$ where we can hope that both $n_1$ and $n_2$ are honest DHT nodes (since the DHT keys are somewhat randomized). This is not the case for larger $\delta$. Note that malicious DHT nodes can't add records to the blacklist without solving the POW but can hide the existing records.

To avoid that, we can store $N$ extra copies of the two records above with slightly modified keys:

$key' = key \, \text{"_copy_"} \, i, \quad i = 1, ..., N$

We can hope that both $n_1$ and $n_2$ are honest at least for one copy (probability of it increase with increasing $N$).

Then, while checking whether someone is in blacklist, we can check for the copies as well.